In [1]:
import numpy as np
import numpy.linalg as la

In [5]:
def dual_simplex_iteration(y, B, problem, verbose=False):
    """Perform one dual simplex iteration given 
    - basic dual feasible solution y
    - basis B
    
    It returns new y, new basis and termination flag (true/false)
    """  
    A, b, c = problem['A'], problem['b'], problem['c']
    m, n = A.shape
    N = np.setdiff1d(np.arange(n), B)
    A_B = A[:, B]
    A_N = A[:, N]
    c_bar = c + A.T @ y
    
    if verbose:
        print("y =", y)
        print("B =", B)
        print("-b' * y =", -b @ y)
        print("A_B =\n", A_B)
        print("c_bar =", c_bar)
        
    # Compute primal feasible solution vector
    x = np.zeros(n)
    x[B] = la.solve(A_B, b)
    if verbose:
        print("x", x)   
        
    # Check optimality
    if np.all(x >= 0):
        if verbose:
            print("Optimal solution found!")
        return y, B, True, x
    
    # Choose j such that x_j < 0 (first one)
    # Exiting the basis
    i = np.where(x < 0)[0][0]
    if verbose:
        print("i = ", i)
        
    # Compute search direction z
    z = np.zeros(n)
    z[i] = 1
    d = la.solve(A_B.T, z[B])
    z[N] = A_N.T @ d
    if verbose:
        print("z =", z)
        print("d =", d)
    
    # Check for unboundedness
    if np.all(z >= 0):
        if verbose:
            print("Unbounded problem!")
        return None, None, True, None
    
    # Compute step length theta
    z_j = np.where(z[N] < 0)[0]
    theta = np.min(- c_bar[N[z_j]] / z[N[z_j]])
    j = N[z_j[np.argmin(- c_bar[N[z_j]] / z[N[z_j]])]]
    if verbose:
        print("j = ", j)
        print("theta = min_{j | z_j < 0} (-c_bar_j/z_j) = min(", - c_bar[N[z_j]] / z[N[z_j]], ") =", theta)

    # Compute next point
    y_next = y + theta * d
    if verbose:
        print("y_next = y + theta * d =", y_next)    
    
    # Compute next basis
    B_next = B
    B_next[np.where(B == i)[0]] = j    
  
    return y_next, B_next, False, None   

    
def dual_simplex_algorithm(y, B, problem, max_iter=1000, verbose=False):

    for k in range(max_iter):
        
        if verbose:
            print("\nIteration", k)
            
        y, B, end, x = dual_simplex_iteration(y, B, problem, verbose=verbose)
         
        if end:
            break
        
        input("Press Enter to continue...")
    return y, B, x

In [6]:
# Example primal simplex
c = np.array([-10, -12, -12, 0, 0, 0])
A = np.array([[1, 2, 2, 1, 0, 0],
              [2, 1, 2, 0, 1, 0],
              [2, 2, 1, 0, 0, 1]])
b = np.array([20, 20, 20])

# Initial point
B0 = np.array([0, 4, 5])
y0 = np.array([10, 0, 0])

In [7]:
y_simplex, B_simplex, x_simplex = dual_simplex_algorithm(y0, B0, {'c': c, 'A': A, 'b': b}, verbose=True)


Iteration 0
y = [10  0  0]
B = [0 4 5]
-b' * y = -200
A_B =
 [[1 0 0]
 [2 1 0]
 [2 0 1]]
c_bar = [ 0  8  8 10  0  0]
x [ 20.   0.   0.   0. -20. -20.]
i =  4
z = [ 0. -3. -2. -2.  1.  0.]
d = [-2.  1.  0.]
j =  1
theta = min_{j | z_j < 0} (-c_bar_j/z_j) = min( [2.66666667 4.         5.        ] ) = 2.6666666666666665
y_next = y + theta * d = [4.66666667 2.66666667 0.        ]


Press Enter to continue... 



Iteration 1
y = [4.66666667 2.66666667 0.        ]
B = [0 1 5]
-b' * y = -146.66666666666669
A_B =
 [[1 2 0]
 [2 1 0]
 [2 2 1]]
c_bar = [0.         0.         2.66666667 4.66666667 2.66666667 0.        ]
x [ 6.66666667  6.66666667  0.          0.          0.         -6.66666667]
i =  5
z = [ 0.          0.         -1.66666667 -0.66666667 -0.66666667  1.        ]
d = [-0.66666667 -0.66666667  1.        ]
j =  2
theta = min_{j | z_j < 0} (-c_bar_j/z_j) = min( [1.6 7.  4. ] ) = 1.6000000000000005
y_next = y + theta * d = [3.6 1.6 1.6]


Press Enter to continue... 



Iteration 2
y = [3.6 1.6 1.6]
B = [0 1 2]
-b' * y = -136.0
A_B =
 [[1 2 2]
 [2 1 2]
 [2 2 1]]
c_bar = [0.  0.  0.  3.6 1.6 1.6]
x [4. 4. 4. 0. 0. 0.]
Optimal solution found!
